In [13]:
import cv2
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load pre-trained models
cnn_model = load_model('cnn_fruit_model.h5')
ann_model = load_model('ann_calorie_model.h5')

# Load Label Binarizer for decoding predictions
with open('label_binarizer.pkl', 'rb') as file:
    lb = pickle.load(file)

# Load Excel file with nutritional data
data = pd.read_csv('fruits.csv')

# Initialize scaler (same scaler used during ANN model training)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Fit the scaler with nutritional data (excluding Fruit Name column)
features = data[['water (g)',	'protein (g)',	'total fat (g)',	'carbohydrates (g)',	'fiber (g)'	,'sugars (g)'	,'calcium (mg)',	'iron (mg)',	'magnessium (mg)',	'phosphorus (mg)',	'potassium (mg)',	'sodium (g)']]
calories = data[['energy (kcal/kJ)']]
scaler_X.fit(features)
scaler_y.fit(calories)

# Preprocess input image
def preprocess_image(image_path):
    IMG_SIZE = (100, 100)
    image = cv2.imread(image_path)
    image = cv2.resize(image, IMG_SIZE)
    image = image / 255.0
    return image.reshape(1, 100, 100, 3)

@tf.function
def predict_fruit(image):
  prediction = cnn_model.predict(image)  # Get the prediction
  return prediction[0]  # Return the first element (prediction for single image)

# Define a function for predicting diet recommendation


def predict_diet(image_path):
    # Step 1: Preprocess the image
    image = preprocess_image(image_path)

    # Step 2: Classify the fruit using CNN model
    fruit_pred = np.array(cnn_model.predict(image))  # Convert to NumPy array
    fruit_label_idx = np.argmax(fruit_pred)  # Find the index of the highest prediction score
    fruit_name = lb.classes_[fruit_label_idx]  # Get the fruit name from the label binarizer

    # Step 3: Retrieve nutritional data based on predicted fruit label
    if fruit_name not in data['name'].values:
        return f"Error: '{fruit_name}' not found in dataset. Please ensure folder names and Excel entries are consistent."

    fruit_data = data[data['name'] == fruit_name]
    features = fruit_data[['water (g)',	'protein (g)',	'total fat (g)',	'carbohydrates (g)',	'fiber (g)'	,'sugars (g)'	,'calcium (mg)',	'iron (mg)',	'magnessium (mg)',	'phosphorus (mg)',	'potassium (mg)',	'sodium (g)']].values
    features_scaled = scaler_X.transform(features)

    # Step 4: Refine calorie prediction using ANN model
    calorie_prediction_scaled = ann_model.predict(features_scaled)
    refined_calorie_value = scaler_y.inverse_transform(calorie_prediction_scaled)[0][0]

    # Step 5: Return predicted fruit and detailed nutritional information
    return {
        'Fruit Name': fruit_name,
        'Calorie (kcal)': round(refined_calorie_value, 2),
        'Water (g)': fruit_data['water (g)'].values[0],
        'Protein (g)': fruit_data['protein (g)'].values[0],
        'Fiber (g)': fruit_data['fiber (g)'].values[0],
        'Sugar (g)': fruit_data['sugars (g)'].values[0],
        'Carbohydrates (g)': fruit_data['carbohydrates (g)'].values[0],
        'Total fat (g)': fruit_data['total fat (g)'].values[0],
        'Sugars (g)': fruit_data['sugars (g)'].values[0],
        'Calcium (mg)': fruit_data['calcium (mg)'].values[0],
        'Iron (mg)': fruit_data['iron (mg)'].values[0],
        'Magnesium (mg)': fruit_data['magnessium (mg)'].values[0],
        'Phosphorus (mg)': fruit_data['phosphorus (mg)'].values[0],
        'Potassium (mg)': fruit_data['potassium (mg)'].values[0],
        'Sodium (g)': fruit_data['sodium (g)'].values[0]
    }


In [16]:
result = predict_diet('watermelon1.jpg')
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
{'Fruit Name': 'Watermelon 1', 'Calorie (kcal)': 20.43, 'Water (g)': 91.45, 'Protein (g)': 0.61, 'Fiber (g)': 0.4, 'Sugar (g)': 6.2, 'Carbohydrates (g)': 7.55, 'Total fat (g)': 0.15, 'Sugars (g)': 6.2, 'Calcium (mg)': 7.0, 'Iron (mg)': 0.24, 'Magnesium (mg)': 10.0, 'Phosphorus (mg)': 11, 'Potassium (mg)': 112, 'Sodium (g)': 1.0}


c:\Users\Acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [17]:
result = predict_diet('pear.jpg')
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
{'Fruit Name': 'Pear 1', 'Calorie (kcal)': 52.72, 'Water (g)': 83.71, 'Protein (g)': 0.38, 'Fiber (g)': 3.1, 'Sugar (g)': 9.8, 'Carbohydrates (g)': 15.46, 'Total fat (g)': 0.12, 'Sugars (g)': 9.8, 'Calcium (mg)': 9.0, 'Iron (mg)': 0.17, 'Magnesium (mg)': 7.0, 'Phosphorus (mg)': 11, 'Potassium (mg)': 119, 'Sodium (g)': 1.0}


c:\Users\Acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
